In [6]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

In [11]:
df = pd.read_csv("cleaned_data/stacked_data.csv")
df

,category,subcategory,country,year,value
0,Animals,"Bovine animals, live",Canada,2023,1311.1
1,Animals,"Bovine animals, live",Canada,2022,1132.0
2,Animals,"Bovine animals, live",Canada,2021,859.2
3,Animals,"Bovine animals, live",Canada,2020,872.7
4,Animals,"Bovine animals, live",Canada,2019,956.8
...,...,...,...,...,...
14445,Vegetables,Total,Spain,2019,254.9
14446,Vegetables,Total,Spain,2020,231.3
14447,Vegetables,Total,Spain,2021,240.8
14448,Vegetables,Total,Spain,2022,280.7


In [23]:
# Initialize the app
app = Dash()

# App layout
app.layout = html.Div([
    html.H1(children='Comparison of Imports Across Countries and Sub-Categories',
            style={
            'color': 'black',           # Text color
            'backgroundColor': 'white',  # Background color
            'textAlign': 'center',     # Center-align text
            'fontSize': '24px',        # Font size
            'padding': '10px'          # Padding around the text
        }),
    
    
    # Dropdown for countries
    dcc.Dropdown(
        options=[{'label': c, 'value': c} for c in df['country'].unique()],
        value=['Canada'],  # Default selection
        id='dropdown-selection-country',
        multi=True,  # Enable multiple selection
        placeholder="Select one or more countries"
    ),
    
    # Dropdown for categories
    dcc.Dropdown(
        options=[{'label': c, 'value': c} for c in df['category'].unique()],
        value=['Animals'],  # Default selection
        id='dropdown-selection-category',
        multi=True,  # Enable multiple selection
        placeholder="Select one or more categories"
    ),
    
    # Time slider for years
    dcc.Slider(
        min=df['year'].min(),
        max=df['year'].max(),
        step=1,
        marks={year: str(year) for year in range(df['year'].min(), df['year'].max() + 1)},
        value=df['year'].max(),  # Default to the most recent year
        id='slider-year-selection'
    ),
    
    # Tabs for categories
    dcc.Tabs(id='tabs-categories', children=[]),
])

# Callback to update tabs and graphs based on selections
@callback(
    Output('tabs-categories', 'children'),
    [
        Input('dropdown-selection-country', 'value'),
        Input('dropdown-selection-category', 'value'),
        Input('slider-year-selection', 'value'),
    ]
)
def update_tabs(selected_countries, selected_categories, selected_year):
    # Filter data by year
    filtered_df = df[df['year'] == selected_year]
    
    # Generate one tab per selected category
    tabs = []
    for category in selected_categories:
        dff = filtered_df[(filtered_df['country'].isin(selected_countries)) & (filtered_df['category'] == category)]
        fig = px.bar(
            dff,
            x='subcategory',
            y='value',
            color='country',
            title=f'{category} Imports in {selected_year} ({", ".join(selected_countries)})',
            labels={'value': 'Millions of Dollars', 'subcategory': 'Sub-Categories'},
            barmode='group'
        )
        tabs.append(dcc.Tab(label=category, children=[
            dcc.Graph(figure=fig)
        ]))
    return tabs

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
